# Taller Big Data - Polars
 Hans Rodríguez Cantor  
 Yulieth Velandia Rojas


## Introducción
Este taller tiene como objetivo hacer uso de las librería polars en python  para realizar diferentes tipos de análisis en la data obtenida del censo nacional de población y vivienda realizado en el año 2018. La información presentada es del municipio de Bogotá.

### Carga de información y librerías

In [18]:
!pip install polars

In [19]:
!python --version

Python 3.9.13


In [20]:
!pip show polars

Name: polars
Version: 0.17.15
Summary: Blazingly fast DataFrame library
Home-page: 
Author: 
Author-email: Ritchie Vink <ritchie46@gmail.com>
License: 
Location: c:\users\yulie\anaconda3\lib\site-packages
Requires: 
Required-by: 


In [21]:
!pip install xlsx2csv

In [22]:
import polars as pl

In [23]:
import pyarrow as pa
import os
import pandas as pd

In [24]:
os.chdir(r"C:\Users\yulie\OneDrive\Documentos\Semestre 3\Big Data\Censo_unificado\parquet")
os.listdir()
df_personas_lz = pl.scan_parquet("personas/*.parquet")
df_viviendas_lz=pl.scan_parquet("viviendas/*.parquet")
df_hogares_lz=pl.scan_parquet("hogares/*.parquet")

### 1. Calcular la proporción de indígenas, afrocolombianos por área urbana y rural
Para realizar este análisis, se hace uso de la variable UA_CLASE de la base de viviendas, que es la que nos presenta la información de zona. Debido a que presenta 4 opciones, se parametriza para agrupar los valores de las variables en urbana o rural.

In [25]:
grupo_etnia = pl.read_excel("C:\\Users\\yulie\\OneDrive\\Documentos\\Semestre 3\\Big Data\\Censo_unificado\\Grupo_etnico.xlsx")
tipo_poblacion = pl.read_excel("C:\\Users\\yulie\\OneDrive\\Documentos\\Semestre 3\\Big Data\\Censo_unificado\\Tipo_Población.xlsx")

Se realizan operaciones join para unir las tablas de vivienda y personas con sus respectivos diccionarios de datos para determinar el tipo de área y el grupo étnico.

In [26]:
consulta1 = df_viviendas_lz.select("U_DPTO","COD_ENCUESTAS", "UA_CLASE").filter((pl.col("U_DPTO") == "11")).collect()
consulta1_area=consulta1.join(tipo_poblacion, on="UA_CLASE")
consulta2 = df_personas_lz.select("U_DPTO","COD_ENCUESTAS","PA1_GRP_ETNIC").filter((pl.col("U_DPTO") == "11")).collect()
consulta2_etnia=consulta2.join(grupo_etnia, on="PA1_GRP_ETNIC")
areaxetnia = consulta2_etnia.join(consulta1_area, on="COD_ENCUESTAS")

Definido lo anterior, se procede a hacer el pipeline que permite identificar cual es la cantidad de población indígenas y afrocolombiana del área urbana y del área rural.

In [27]:
area_etnia= areaxetnia.select("COD_ENCUESTAS","PA1_GRP_ETNIC","Nombre_Grupo","Zona").filter((pl.col("PA1_GRP_ETNIC")==1)|(pl.col("PA1_GRP_ETNIC")==5)).groupby(["Zona","Nombre_Grupo"]).agg(pl.count("COD_ENCUESTAS").alias("Total"))
area_etnias=area_etnia.pivot("Total", index="Zona", columns="Nombre_Grupo",aggregate_function="first")
area_etnias=pl.DataFrame.to_pandas(area_etnias)

Finalmente, se agrega la proporción de población indigena y afrodescendiente.

In [28]:
# Calcular las sumas totales
total_indigena = area_etnias["\tIndígena"].sum()
total_afrodescendiente = area_etnias["Afrodescendiente"].sum()

# Calcular las proporciones
area_etnias["Propoción Indígena"] = area_etnias["\tIndígena"] / total_indigena * 100
area_etnias["Proporción Afrodescendiente"] = area_etnias["Afrodescendiente"] / total_afrodescendiente * 100

# Formato de porcentaje
area_etnias["Propoción Indígena"] = area_etnias["Propoción Indígena"].map("{:.2f}%".format)
area_etnias["Proporción Afrodescendiente"] = area_etnias["Proporción Afrodescendiente"].map("{:.2f}%".format)

area_etnias

,Zona,Afrodescendiente,\tIndígena,Propoción Indígena,Proporción Afrodescendiente
0,Urbana,65538,18983,99.58%,99.82%
1,Rural,118,80,0.42%,0.18%


### 2. Utilice la tabla de viviendas y calcule la tabla de frecuencias y relativas

Para este análisis se debe hacer uso los dos grupos de datos correspondientes a viviendas y personas.

In [29]:
consulta_estrato = df_viviendas_lz.select("COD_ENCUESTAS","VA1_ESTRATO","U_DPTO").filter((pl.col("U_DPTO") == "11")).collect()
q_personas=df_personas_lz.select("COD_ENCUESTAS","U_DPTO").filter((pl.col("U_DPTO") == "11")).collect()
consulta_est=consulta_estrato.join(q_personas, on="COD_ENCUESTAS")

In [30]:
consulta_estratos = (consulta_est.select("VA1_ESTRATO").filter((pl.col("VA1_ESTRATO") != 9) & (pl.col("VA1_ESTRATO") != 0) & (~pl.col("VA1_ESTRATO").is_null()))
    .groupby("VA1_ESTRATO").agg(pl.count()).sort("VA1_ESTRATO", descending=True))

In [31]:
consulta_estratos3=pl.DataFrame.to_pandas(consulta_estratos)

In [32]:
# Calcular la suma total de la columna "count"
total = consulta_estratos3["count"].sum()

# Calcular la columna "Frecuencia Absoluta"
consulta_estratos3["Frecuencia Absoluta"] = consulta_estratos3["count"].cumsum()

# Calcular la columna "Frecuencia Relativa"
consulta_estratos3["Frecuencia Relativa"] = consulta_estratos3["Frecuencia Absoluta"] / total * 100
consulta_estratos3["Frecuencia Relativa"] = consulta_estratos3["Frecuencia Relativa"].map("{:.0f}%".format)

consulta_estratos3


,VA1_ESTRATO,count,Frecuencia Absoluta,Frecuencia Relativa
0,6.0,149073,149073,2%
1,5.0,211459,360532,5%
2,4.0,672543,1033075,15%
3,3.0,2411701,3444776,48%
4,2.0,3002092,6446868,91%
5,1.0,671690,7118558,100%


### 3. Calcule la proporción de bogotanos mayores (estricto) de 25 años que han alcanzado el siguiente máximo nivel educativo: primaria o menos, bachiller,  técnica profesional/tecnológica o educación superior o posgrado:

In [33]:
consulta_educacion = df_personas_lz.select("U_DPTO","P_NIVEL_ANOSR", "P_EDADR").filter((pl.col("U_DPTO") == "11")&(pl.col("P_EDADR") >6)&(pl.col("P_NIVEL_ANOSR") != 99)&(pl.col("P_NIVEL_ANOSR") != 10)).groupby("P_NIVEL_ANOSR").agg(pl.count()).collect()

In [34]:
categoria=tipo_poblacion = pl.read_excel("C:\\Users\\yulie\\OneDrive\\Documentos\\Semestre 3\\Big Data\\Censo_unificado\\Nivel_educativo.xlsx")

In [35]:
nivel=consulta_educacion.join(categoria, on="P_NIVEL_ANOSR")

In [36]:
total_nivel = nivel.groupby("Grupos").agg(pl.sum("count").alias("Total_nivel"))
total_nivel_df=pl.DataFrame.to_pandas(total_nivel)

In [37]:
# Calcular el total
total = total_nivel_df["Total_nivel"].sum()

# Calcular la columna "Proporción"
total_nivel_df["Proporción"] = total_nivel_df["Total_nivel"] / total * 100
total_nivel_df["Proporción"] = total_nivel_df["Proporción"].map("{:.0f}%".format)

total_nivel_df

,Grupos,Total_nivel,Proporción
0,Bachiller,1451412,39%
1,Primaria o menos,774871,21%
2,Educación Superior,1509993,40%


### 4. Calcule el número de personas por hogar por nivel socioeconómico
El siguiente análisis requiere usar el conjunto relacionado a hogares y el relacionado a viviendas, para finalmente calcular el promedio de personas por hogar en cada estrato.

In [38]:
consulta_hogares=df_hogares_lz.select("COD_ENCUESTAS","U_DPTO","HA_TOT_PER").filter((pl.col("U_DPTO") == "11")).collect()
consulta_pxh=consulta_estrato.join(consulta_hogares, on="COD_ENCUESTAS")

In [39]:
consulta_pxhogar = (consulta_pxh.select("VA1_ESTRATO","HA_TOT_PER").filter((pl.col("VA1_ESTRATO") != 9) & (pl.col("VA1_ESTRATO") != 0) & (~pl.col("VA1_ESTRATO").is_null()))
    .groupby("VA1_ESTRATO").agg(pl.col("HA_TOT_PER").mean()).sort("VA1_ESTRATO", descending=True))

In [40]:
consulta_pxhogar

VA1_ESTRATO,HA_TOT_PER
f64,f64
6.0,2.420684
5.0,2.517759
4.0,2.503305
3.0,2.809102
2.0,2.973782
1.0,3.054316


### 5. Calcular la siguiente tabla:

Para hacer el cálculo de la tabla solicitada, se realiza un join de las tablas de personas y viviendas por el ID de la encuesta:

In [41]:
consulta_edu2= df_personas_lz.select("U_DPTO","P_NIVEL_ANOSR", "COD_ENCUESTAS").filter((pl.col("U_DPTO") == "11")&(pl.col("P_NIVEL_ANOSR") != 99)&(pl.col("P_NIVEL_ANOSR") != 10))
consulta_estrato2 = df_viviendas_lz.select("COD_ENCUESTAS","VA1_ESTRATO","U_DPTO").filter((pl.col("U_DPTO") == "11"))
nivelxestrato=consulta_edu2.join(consulta_estrato2, on="COD_ENCUESTAS").collect()

In [42]:
estrato = pl.read_excel("C:\\Users\\yulie\\OneDrive\\Documentos\\Semestre 3\\Big Data\\Censo_unificado\\Estratos.xlsx")
consulta_nxe=nivelxestrato.join(categoria, on="P_NIVEL_ANOSR")
consulta_nxe2=consulta_nxe.join(estrato, on="VA1_ESTRATO")
total_nxe3 = consulta_nxe2.select("ESTRATO","VA1_ESTRATO","COD_ENCUESTAS","Grupos").filter((pl.col("VA1_ESTRATO") != 9) & (pl.col("VA1_ESTRATO") != 0) & (~pl.col("VA1_ESTRATO").is_null())).groupby(["Grupos", "ESTRATO"]).agg(pl.count("COD_ENCUESTAS").alias("Total"))
total_nxe3 = total_nxe3.sort("ESTRATO", descending=True)

In [43]:
Tabla_total = total_nxe3.pivot("Total", index="Grupos",columns="ESTRATO",aggregate_function="first")
Tabla_total_df=pl.DataFrame.to_pandas(Tabla_total)

El resultado del análisis realizado muestra la siguiente tabla, donde se muestra la frecuencia del nivel educativo (Frec_est) y la proporción de la misma (Freq_r_)

In [44]:
# Calcular la sumatoria acumulada de cada columna de estrato
for i in range(6, 0, -1):
    column_name = f'Estrato_{i}'
    cumulative_sum = Tabla_total_df[column_name].cumsum()
    Tabla_total_df[f'Frecuencia absoluta e{i}'] = cumulative_sum

# Calcular la frecuencia relativa para cada columna de estrato
for i in range(6, 0, -1):
    column_name = f'Frecuencia absoluta e{i}'
    total_sum = Tabla_total_df[f'Estrato_{i}'].sum()
    Tabla_total_df[f'Frecuencia relativa e{i}'] = Tabla_total_df[column_name] / total_sum

nuevo_orden = ['Grupos', 'Estrato_6','Frecuencia absoluta e6', 'Frecuencia relativa e6',
               'Estrato_5','Frecuencia absoluta e5', 'Frecuencia relativa e5',
               'Estrato_4','Frecuencia absoluta e4', 'Frecuencia relativa e4',
               'Estrato_3','Frecuencia absoluta e3', 'Frecuencia relativa e3',
               'Estrato_2','Frecuencia absoluta e2', 'Frecuencia relativa e2',
               'Estrato_1','Frecuencia absoluta e1','Frecuencia relativa e1']    
Tabla_total_df = Tabla_total_df.reindex(columns=nuevo_orden)


Tabla_total_df

,Grupos,Estrato_6,Frecuencia absoluta e6,Frecuencia relativa e6,Estrato_5,Frecuencia absoluta e5,Frecuencia relativa e5,Estrato_4,Frecuencia absoluta e4,Frecuencia relativa e4,Estrato_3,Frecuencia absoluta e3,Frecuencia relativa e3,Estrato_2,Frecuencia absoluta e2,Frecuencia relativa e2,Estrato_1,Frecuencia absoluta e1,Frecuencia relativa e1
0,Educación Superior,103154,103154,0.754309,144597,144597,0.745699,437034,437034,0.708005,966444,966444,0.437356,632918,632918,0.234235,74897,74897,0.127382
1,Bachiller,21613,124767,0.912353,32667,177264,0.914165,122560,559594,0.906555,841462,1807906,0.818153,1337943,1970861,0.729391,302657,377554,0.642128
2,Primaria o menos,11986,136753,1.000000,16644,193908,1.000000,57681,617275,1.000000,401834,2209740,1.000000,731204,2702065,1.000000,210419,587973,1.000000


¿En qué estrato socioeconómico está la gente más educada? Las personas con Educación Superior o Posgrado están principalmente en los estratos 5 y 6.